Clase 6
===

* Ajustar modelo a variables experimentales
* Encontrar la solución más semejante a datos experimentales


Medir calidad de las simulaciones
--------------------------------------

Si conocemos el valor de uno o varios flujos metabólicos podemos ajustar nuestro modelo a estos datos. Hay varias maneras de hacer esto, la primera es simplemente fijar el valor de los flujos como veremos a continuación. 

A modo de ejemplo usaremos *Saccharomyces cerevisae* iMM904 [(descargar aquí)](http://bigg.ucsd.edu/models/iMM904) y los flujos experimentales compilados por [García et al 2012](https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0043006)


In [2]:
# Let's import the yeast model
import cobra
model = cobra.io.read_sbml_model("iMM904.xml")

In [3]:
# And check the solution using default parameters
solution=model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.611,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05691,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.64,0,0.00%


In [5]:
# Now, let's modify the ethanol flux
# First, lets find the reactions associated with ethanol
import re

def buscarMetabolito(nombreMetabolite):
    metabolites=[]
    for metabolite in model.metabolites:
        if re.match(nombreMetabolite,metabolite.name,re.IGNORECASE):# and re.match("_e",metabolite.name): 
            metabolites.append(metabolite)
    if len(metabolites)==0:
        print( "Not found metabolite" )
        return None
    else: #print "2 lets find the reactions associated with the name"
        for metabolite in metabolites:
            print( "==================================")
            print( metabolite.name, metabolite.id)
            print( "==================================")
            for reaction in model.metabolites.get_by_id(metabolite.id).reactions:
                print( reaction )
result=buscarMetabolito("^ethanol$")

Ethanol etoh_c
ACHLE3: aces_c + h2o_c --> ac_c + etoh_c + h_c
OHACT1: accoa_c + etoh_c --> aces_c + coa_c
ETOHtm: etoh_c <=> etoh_m
ALCD2x_copy1: etoh_c + nad_c --> acald_c + h_c + nadh_c
ETOHt: etoh_e <=> etoh_c
ALCD2x_copy2: etoh_c + nad_c <=> acald_c + h_c + nadh_c
ALCD2ir: acald_c + h_c + nadh_c --> etoh_c + nad_c
Ethanol etoh_e
ETOHt: etoh_e <=> etoh_c
EX_etoh_e: etoh_e --> 
Ethanol etoh_m
ALCD2irm: acald_m + h_m + nadh_m --> etoh_m + nad_m
ETOHtm: etoh_c <=> etoh_m


In [6]:
# Fix the ethanol reaction flux and see how it affects the solution
# Fix a range for the ethanol reaction flux around +/-10% of the observed value
value=3.632
model.reactions.get_by_id("EX_etoh_e").upper_bound=value+0.1*value 
model.reactions.get_by_id("EX_etoh_e").lower_bound=value-0.1*value

solution=model.optimize()
model.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.382,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.04881,0,0.00%
so4_e,EX_so4_e,0.01908,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
btd_RR_e,EX_btd_RR_e,-4.81,4,37.63%
co2_e,EX_co2_e,-11.88,1,23.25%
etoh_e,EX_etoh_e,-3.995,2,15.63%
for_e,EX_for_e,-0.001276,1,0.00%


Es posible que errores de medición resulten en datos de flujos metabólicos que no son estequiometricamente compatibles. Para estos casos, una alternativa es fijar un rango de valores en lugar de un valor exacto. Por ejemplo:

Ejercicio 1
---------

Usando los valores experimentales compilados por [García et al 2012](https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0043006) desde la referencia [20] (ver la columna **References** en Table_S1.doc):

1. Fija la reacción de consumo de glucosa, succinato, ethanol, y piruvato a datos experimentales (haciendo 1 esto debería ser trivial).

Evaluación de la calidad de las predicciones
------------------

Para evaluar que tan cercana es la simulación de FBA a los datos experimentales podemos usar la norma Euclideana la cual mide la distancia entre dos puntos en el hiperespacio:
    
\begin{align}
    \mbox{Norma Euclideana}=\sum_i (x_i-y_i)^2
\end{align}

La cual puede ser escrita usando notación vectorial:

\begin{align}
    \mbox{Norma Euclideana} = (x-y)^T(x-y)
\end{align}


Ejercicio 2
-----------

Escribe una función para determinar la distancia entre los datos simulados y los experimentales (usa los datos de la Ref 20) usando la norma Euclideana como métrica.
